In [108]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import mlbgame
from __future__ import print_function

import datetime
#imports

In [26]:
batters_df = pd.read_csv('../data/individual_batting_stats.csv')
pitchers_df = pd.read_csv('../data/individual_pitching_stats.csv')

In [27]:
batters_df.columns

Index(['last_name', ' first_name', 'player_id', 'year', 'b_bb_percent',
       'batting_avg', 'slg_percent', 'xba', 'xslg', 'woba', 'xwoba', 'xobp',
       'xiso', 'wobacon', 'xwobacon', 'bacon', 'xbacon', 'xbadiff', 'xslgdiff',
       'wobadif', 'exit_velocity_avg', 'launch_angle_avg',
       'sweet_spot_percent', 'barrel_batted_rate', 'Unnamed: 24'],
      dtype='object')

In [28]:
pitchers_df.columns

Index(['last_name', ' first_name', 'player_id', 'year', 'p_bb_percent',
       'batting_avg', 'slg_percent', 'xba', 'xslg', 'woba', 'xwoba', 'xobp',
       'xiso', 'wobacon', 'xwobacon', 'bacon', 'xbacon', 'xbadiff', 'xslgdiff',
       'wobadif', 'exit_velocity_avg', 'launch_angle_avg',
       'sweet_spot_percent', 'barrel_batted_rate', 'Unnamed: 24'],
      dtype='object')

In [29]:
batters_df.describe()

,player_id,year,b_bb_percent,batting_avg,slg_percent,xba,xslg,woba,xwoba,xobp,...,bacon,xbacon,xbadiff,xslgdiff,wobadif,exit_velocity_avg,launch_angle_avg,sweet_spot_percent,barrel_batted_rate,Unnamed: 24
count,175.000000,175.0,175.000000,175.000000,175.000000,175.000000,175.000000,175.000000,175.000000,175.000000,...,175.000000,175.000000,175.000000,175.000000,175.000000,175.000000,175.000000,175.000000,175.000000,0.0
mean,604565.645714,2022.0,9.220571,0.241611,0.387697,0.257914,0.448177,0.318137,0.338023,0.335354,...,0.318554,0.344520,-0.016303,-0.060480,-0.019886,89.158857,13.241714,34.038286,8.410286,NaN
std,62991.004154,0.0,4.633019,0.062356,0.128633,0.048032,0.111385,0.074293,0.059298,0.056054,...,0.072156,0.052933,0.045457,0.088917,0.049066,2.869543,6.419450,8.074671,5.625252,NaN
min,408234.000000,2022.0,0.000000,0.083000,0.104000,0.144000,0.189000,0.138000,0.186000,0.188000,...,0.121000,0.232000,-0.138000,-0.316000,-0.157000,80.400000,-2.000000,15.400000,0.000000,NaN
25%,571714.000000,2022.0,5.900000,0.196000,0.298000,0.230000,0.372000,0.268000,0.300000,0.298000,...,0.274000,0.307000,-0.048000,-0.111500,-0.052000,87.400000,8.950000,27.550000,4.050000,NaN
50%,621035.000000,2022.0,8.900000,0.238000,0.373000,0.256000,0.438000,0.312000,0.332000,0.335000,...,0.310000,0.344000,-0.018000,-0.066000,-0.020000,89.400000,13.200000,35.300000,7.300000,NaN
75%,660491.000000,2022.0,12.600000,0.275500,0.471500,0.283000,0.515500,0.375500,0.375500,0.369500,...,0.360500,0.374000,0.010000,-0.001000,0.013000,90.800000,17.150000,39.350000,12.200000,NaN
max,680757.000000,2022.0,22.100000,0.388000,0.787000,0.393000,0.842000,0.507000,0.532000,0.476000,...,0.515000,0.499000,0.136000,0.183000,0.111000,96.500000,32.300000,57.100000,26.200000,NaN


In [30]:
batters_df.columns

Index(['last_name', ' first_name', 'player_id', 'year', 'b_bb_percent',
       'batting_avg', 'slg_percent', 'xba', 'xslg', 'woba', 'xwoba', 'xobp',
       'xiso', 'wobacon', 'xwobacon', 'bacon', 'xbacon', 'xbadiff', 'xslgdiff',
       'wobadif', 'exit_velocity_avg', 'launch_angle_avg',
       'sweet_spot_percent', 'barrel_batted_rate', 'Unnamed: 24'],
      dtype='object')

In [31]:
batters_df['first_name'] = batters_df[' first_name']
pitchers_df['first_name'] = pitchers_df[' first_name']
#Existing column has an extra space in the front so replacing it with new column name

In [32]:
bat_xbadiff = batters_df[['last_name', 'first_name', 'xbadiff','batting_avg', 'xba']]
#Make a table with just the xBA difference

In [69]:
bat_xbadiff.sort_values('xba', ascending=False)[0:30]

,last_name,first_name,xbadiff,batting_avg,xba
143,France,Ty,-0.018,0.375,0.393
170,Franco,Wander,-0.042,0.349,0.391
56,Pederson,Joc,-0.029,0.362,0.391
73,Bell,Josh,-0.025,0.345,0.370
90,Olson,Matt,-0.001,0.355,0.356
21,LeMahieu,DJ,-0.021,0.333,0.354
103,Mancini III,Trey,-0.109,0.237,0.346
100,Crawford,J.P.,0.007,0.352,0.345
10,Brantley Jr.,Michael,-0.031,0.311,0.342
17,Segura,Jean,-0.069,0.265,0.334


In [71]:
pitch_xbadiff = pitchers_df[['last_name', 'first_name', 'xbadiff', 'batting_avg', 'xba']]
pitch_xbadiff.sort_values('xba', ascending=False)

,last_name,first_name,xbadiff,batting_avg,xba
40,Plesac,Zach,-0.080,0.258,0.338
43,Adon,Joan,-0.038,0.282,0.320
24,Marquez,German,-0.023,0.292,0.315
26,Berrios,Jose,-0.024,0.289,0.313
27,Flexen,Chris,-0.059,0.246,0.305
11,Hendricks,Kyle,-0.058,0.244,0.302
0,Wainwright,Adam,0.012,0.313,0.301
23,Irvin,Cole,-0.070,0.226,0.296
30,Dunning,Dane,-0.002,0.293,0.295
13,Gausman,Kevin,-0.006,0.288,0.294


# Probable Pitchers

In [66]:
for game in mlbgame.day(2022,4,11):
    print(game)